# Darstellung des Temperaturverlaufes von 24h an einer Wetterstation

Von Lea Pellikan   |   Matrikel-Nr.: 3062311   |   Abgabe: 15.11.2020

Im folgendem Jupyter Notebook soll ein Balkendiagram erstellt werden, das den stündlichen Temperaturverlauf eines ausgewählten Tages an einer ausgewählten Wetterstation darstellt. 

Die dafür verwendeten Daten stammen von der Quelle https://wetterdienst.readthedocs.io/en/latest/pages/api.html. Diese Quelle wird vom Deutschen Wetterdienst (DWD) gepflegt und stellt, unter anderem, die stündlich gemessenen Temperaturen von allen Wetterstationen in Deutschland zur Verfügung.  

Die Daten wurden aus Performanzgründen nicht direkt von Kafka importiert, sondern per CSV-Datei in einer MongoDB gespeichert. Aus der Datenbank werden die benötigten Daten ausgelesen.

## 1. Import der Libraries

Im Folgenden wird die Library `pymongo` für die Verbindung zur MongoDB verwendet und `pandas` für das Verarbeiten der Collection. Außerdem wird `plotly.express` zur Darstellung der Daten verwendet. 

In [5]:
import pandas as pd
import plotly.express as px
from pymongo import MongoClient

## 2. Verbindungsaufbau und Datenauslesung der MongoDB

Der folgende Code baut die Verbindung zur MongoDB auf, wobei die Library `pymongo` verwendet wird, dei Durch Docker realisiert ist.

In [6]:
client = MongoClient('mongodb://localhost:27017/')
stations = client.bda.stations
weather = client.bda.weather

Die methode **get_collection** erstellt ein Dictonary aus der Collection die ausgelesen wurde.

In [7]:
def get_collection(collection):
    documents = []
    cursor = collection.find({})
    for document in cursor:
        documents.append(document)
    return documents

Im Folgenden wird die Methode **get_collection** verwendet, um mit Hilfe von `pandas` das Dataframe *stations_df* mit den Informationen der Wetterstationen und das Dataframe *weather_df* mit den Wetterinformationen zufüllen. Außerdem wird die Spalte *STATION_ID* zu *station_id* umbenannt, um eine einheitliche Benennung zu erhalten.

In [8]:
stations_df = pd.DataFrame(get_collection(stations))
stations_df = stations_df.rename(columns={"STATION_ID": "station_id"})

In [9]:
weather_df = pd.DataFrame(get_collection(weather))

## 3. Mergen und Erläuterung der Daten

Danach erfolgt die Zusammenführung der oben erwähnten Dataframes, wobei die station_id das gemeinsamme Attribut ist. Wie in der Ausgabe zu sehen ist, enthält das Dataframe jetzt folgende Daten:
* die IDs der Wetterstationen (station_id), 
* das Datum inklusive Uhrzeit, zu dem die folgende Temperatur gemessen wurde (date), 
* die Zeitspanne des Datensatzes (FROM_DATE, TO_DATE), 
* das in diesem Beispiel vernachlässigt werden kann, 
* die Höhe der Messstation (STATION_HEIGHT), 
* die Koordinaten der Wetterstation (LAT,LON), 
* den Namen der Station (STATION_NAME) und 
* das Bundesland, in der die Wetterstation sich befindet (STATE).

In [11]:
weather_stations = pd.merge(weather_df,stations_df, on="station_id", how="left")
weather_stations

,_id_x,station_id,date,temperature,_id_y,FROM_DATE,TO_DATE,STATION_HEIGHT,LAT,LON,STATION_NAME,STATE
0,5faaa32e2c9f790f334cc002,44,2020-07-01 00:00:00,14.1,5fac44ab08a1e8a529a9cab2,2007-04-01,2020-11-10,44.0,52.9336,8.2370,Großenkneten,Niedersachsen
1,5faaa32e2c9f790f334cc003,44,2020-07-01 01:00:00,13.7,5fac44ab08a1e8a529a9cab2,2007-04-01,2020-11-10,44.0,52.9336,8.2370,Großenkneten,Niedersachsen
2,5faaa32e2c9f790f334cc004,44,2020-07-01 02:00:00,13.8,5fac44ab08a1e8a529a9cab2,2007-04-01,2020-11-10,44.0,52.9336,8.2370,Großenkneten,Niedersachsen
3,5faaa32e2c9f790f334cc005,44,2020-07-01 03:00:00,13.8,5fac44ab08a1e8a529a9cab2,2007-04-01,2020-11-10,44.0,52.9336,8.2370,Großenkneten,Niedersachsen
4,5faaa32e2c9f790f334cc006,44,2020-07-01 04:00:00,14.0,5fac44ab08a1e8a529a9cab2,2007-04-01,2020-11-10,44.0,52.9336,8.2370,Großenkneten,Niedersachsen
...,...,...,...,...,...,...,...,...,...,...,...,...
717933,5faaa6892c9f790f3357b46f,15555,2020-08-30 20:00:00,9.0,5fac44ab08a1e8a529a9cc9f,2016-05-01,2020-11-10,815.0,47.8761,10.5849,Kaufbeuren-Oberbeuren,Bayern
717934,5faaa6892c9f790f3357b470,15555,2020-08-30 21:00:00,9.0,5fac44ab08a1e8a529a9cc9f,2016-05-01,2020-11-10,815.0,47.8761,10.5849,Kaufbeuren-Oberbeuren,Bayern
717935,5faaa6892c9f790f3357b471,15555,2020-08-30 22:00:00,9.0,5fac44ab08a1e8a529a9cc9f,2016-05-01,2020-11-10,815.0,47.8761,10.5849,Kaufbeuren-Oberbeuren,Bayern
717936,5faaa6892c9f790f3357b472,15555,2020-08-30 23:00:00,9.0,5fac44ab08a1e8a529a9cc9f,2016-05-01,2020-11-10,815.0,47.8761,10.5849,Kaufbeuren-Oberbeuren,Bayern


In der Methode *temp_by_date* wird nach den Temperaturen des übergebenden Datums an der Station, dessen ID übergeben wurde, gefiltert. In der 1. Zeile der Methode wird die Spalte *date* des Daterframes mit dem übergebenen Datum abgeglichen. Da dort die Uhrzeit mit hinterlegt ist, wird diese auf 00:00:00 gesetzt, damit nur nach dem Datum selektiert wird. Das Ergebnis wird danach auf die übergebenen WetterstationsID geprüft.

In [12]:
def temp_by_date(df,date,stationid):
    result= df.loc[df['date'].dt.normalize() == date ]
    result2 =  result.loc[result['station_id']== stationid ]
    return result2

Hier kommt die Methode *temp_by_date* zur Verwendung. Als Beispielparameter wird hier die ID der Wetterstation in Darmstadt (917) und das Datum 20.08.2020 übergeben.
In der Ausgabe ist zu sehen, dass nur noch die Informationen dieser Wetterstation an diesem Datum abgebildet werden. 

(Als alternatives Beispiel empfehle ich Ihnen das Datum des "2020-08-30 00:00:00" zu wählen. Daran ist zu sehen, dass die Höchsttemperatur in den 10 Tagen um ca. 10 Grad gesunken ist.)

In [24]:
tempbydate = temp_by_date(weather_stations,"2020-08-20 00:00:00",917)
tempbydate

,_id_x,station_id,date,temperature,_id_y,FROM_DATE,TO_DATE,STATION_HEIGHT,LAT,LON,STATION_NAME,STATE
100247,5faaa3b32c9f790f334e4799,917,2020-08-20 00:00:00,17.9,5fac44ab08a1e8a529a9caf6,2004-09-01,2020-11-10,162.0,49.8809,8.6779,Darmstadt,Hessen
100248,5faaa3b32c9f790f334e479a,917,2020-08-20 01:00:00,18.1,5fac44ab08a1e8a529a9caf6,2004-09-01,2020-11-10,162.0,49.8809,8.6779,Darmstadt,Hessen
100249,5faaa3b32c9f790f334e479b,917,2020-08-20 02:00:00,18.1,5fac44ab08a1e8a529a9caf6,2004-09-01,2020-11-10,162.0,49.8809,8.6779,Darmstadt,Hessen
100250,5faaa3b32c9f790f334e479c,917,2020-08-20 03:00:00,17.7,5fac44ab08a1e8a529a9caf6,2004-09-01,2020-11-10,162.0,49.8809,8.6779,Darmstadt,Hessen
100251,5faaa3b32c9f790f334e479d,917,2020-08-20 04:00:00,18.2,5fac44ab08a1e8a529a9caf6,2004-09-01,2020-11-10,162.0,49.8809,8.6779,Darmstadt,Hessen
100252,5faaa3b32c9f790f334e479e,917,2020-08-20 05:00:00,17.9,5fac44ab08a1e8a529a9caf6,2004-09-01,2020-11-10,162.0,49.8809,8.6779,Darmstadt,Hessen
100253,5faaa3b32c9f790f334e479f,917,2020-08-20 06:00:00,18.0,5fac44ab08a1e8a529a9caf6,2004-09-01,2020-11-10,162.0,49.8809,8.6779,Darmstadt,Hessen
100254,5faaa3b32c9f790f334e47a0,917,2020-08-20 07:00:00,19.2,5fac44ab08a1e8a529a9caf6,2004-09-01,2020-11-10,162.0,49.8809,8.6779,Darmstadt,Hessen
100255,5faaa3b32c9f790f334e47a1,917,2020-08-20 08:00:00,22.3,5fac44ab08a1e8a529a9caf6,2004-09-01,2020-11-10,162.0,49.8809,8.6779,Darmstadt,Hessen
100256,5faaa3b32c9f790f334e47a2,917,2020-08-20 09:00:00,25.0,5fac44ab08a1e8a529a9caf6,2004-09-01,2020-11-10,162.0,49.8809,8.6779,Darmstadt,Hessen


## 4. Konfiguration des Diagrams

In der Methode *calculate_plot* werden die Daten des oben abgebildete Dataframes in ein Balkendiagram transformiert. Hierbei stellt die X-Achse die Uhrzeit und die Y-Achse die Temperatur der jeweiligen Stunde dar. In der Überschrift wird sowohl das Datum als auch der Name der ausgewählten Wetterstation angezeigt. Die Farbe der Balken ist an die Höhe der Temperatur angepasst. Durch Hovern über die einzelnen Balken wird das Datum, die Temperatur, die StationsID und der Stationsname angezeigt.

In [22]:
def calculate_plot(tempbydate):
    fig =  px.bar(tempbydate,
             y= "temperature", 
             x= "date",
               title = 'Temperaturverlauf am ' + str(tempbydate["date"].dt.date.iloc[0]) +
                ' in ' + str(tempbydate["STATION_NAME"].iloc[0] ) ,
                hover_data=['station_id', 'STATION_NAME'], color='temperature')
    fig.show()

In [23]:
calculate_plot(tempbydate)